# Crear la clase  Target

## Ejercicio 1

Cree el archivo competencia_01.csv, usando el competencia_01_crudo.csv con una variable adicional llamada clase_ternaria, que contenga las categorías **CONTINUA, BAJA+1 y BAJA+2**

Ayudita
Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**.

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction) Procedemos a instalarla, esto se debe ejecutar una sola vez

In [15]:
%pip install duckdb jupysql duckdb-engine pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\jessa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb**

In [16]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Y ya podemos usar **SQL** dentro de una notebook!

In [17]:
%%sql
select 'hola mundo'

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla pasando su ubicación a través de la variable `dataset_path`:

In [18]:
dataset_path = '../datasets/'
dataset_file = 'competencia_01_crudo.csv'

In [19]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [20]:
%sql select * from competencia_01_crudo limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,0.0,4,0.00,-20833.47,0.0,2814,6743.38,6,0,9559.95
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,0.0,11,13267.49,-9662.51,0.0,5541,9419.90,8,0,1900.26
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,0.0,4,0.00,-4692.00,0.0,577,2821.32,2,0,6885.51
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,0.0,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,0.0,4,0.00,-88178.37,0.0,1958,20397.54,8,0,2627.52


In [21]:
%%sql
SELECT
    numero_de_cliente,
    COUNT(*) AS cant_rep
FROM competencia_01_crudo
GROUP BY numero_de_cliente;

,numero_de_cliente,cant_rep
0,918170566,6
1,918183034,6
2,918186100,6
3,918238824,6
4,918250635,6
...,...,...
168876,1596108649,1
168877,1596583543,1
168878,1596885756,1
168879,1596971444,1


In [22]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_01_crudo
group by foto_mes

,foto_mes,cantidad
0,202101,161527
1,202102,162155
2,202103,163109
3,202104,163418
4,202105,163917
5,202106,164313


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide.

In [23]:
%%sql
-- DuckDB: clase_ternaria por continuidad mensual
CREATE OR REPLACE MACRO clase_ternaria(t) AS TABLE (
WITH base AS (
  SELECT
    *,
    -- Índice mensual consecutivo: yyyymm → (yyyy * 12 + mm)
    (CAST(foto_mes / 100 AS INTEGER) * 12 + (foto_mes % 100)) AS period_idx 
  FROM t
),
seq AS (
  SELECT
    *,
    LEAD(period_idx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY period_idx) AS p1,
    LEAD(period_idx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY period_idx) AS p2
  FROM base
),
bounds AS (
  SELECT MAX(period_idx) AS maxp FROM seq
)
SELECT
  * EXCLUDE (period_idx, p1, p2),
  CASE
    -- BAJA+1: falta el mes siguiente (o no vuelve a aparecer)
    WHEN period_idx < (SELECT maxp FROM bounds)
         AND (p1 IS NULL OR p1 > period_idx + 1)
    THEN 'BAJA+1'

    -- BAJA+2: aparece el mes siguiente, pero falta el segundo siguiente
    WHEN period_idx < (SELECT maxp FROM bounds) - 1
         AND p1 = period_idx + 1
         AND (p2 IS NULL OR p2 > period_idx + 2)
    THEN 'BAJA+2'

    -- CONTINUA: meses “antiguos” respecto del borde (al menos 2 meses antes del máximo)
    WHEN period_idx <= (SELECT maxp FROM bounds) - 2
    THEN 'CONTINUA'

    -- Borde (último/anteúltimo mes global): no etiquetamos
    ELSE NULL
  END AS clase_ternaria
FROM seq
);


,Success


In [24]:
%%sql
CREATE OR REPLACE VIEW t AS
SELECT * FROM competencia_01_crudo;

CREATE OR REPLACE TABLE competencia_01 AS
SELECT * FROM clase_ternaria(t);

DROP VIEW t;


,Success


In [25]:
%%sql
-- Chequeo rápido de distribución
SELECT foto_mes, clase_ternaria, COUNT(*) AS n
FROM competencia_01
GROUP BY 1,2
ORDER BY 1,2;

,foto_mes,clase_ternaria,n
0,202101,BAJA+1,622
1,202101,BAJA+2,825
2,202101,CONTINUA,160080
3,202102,BAJA+1,831
4,202102,BAJA+2,1032
5,202102,CONTINUA,160292
6,202103,BAJA+1,1039
7,202103,BAJA+2,951
8,202103,CONTINUA,161119
9,202104,BAJA+1,955


In [26]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,1444378127,202101,1,0,0,29,21,-5809.08,-2632.03,367.85,...,4,0.0,-37206.99,0.0,364,3790.26,3,0,2815.20,CONTINUA
1,1444378127,202102,1,0,0,29,22,333.49,-2691.60,241.59,...,-3,0.0,-27835.39,0.0,392,1173.00,1,0,2932.50,CONTINUA
2,1444378127,202103,1,0,0,29,23,-786.35,-3257.35,96.29,...,1,0.0,-29762.86,0.0,423,6926.33,4,0,3190.56,CONTINUA
3,1444378127,202104,1,0,0,30,24,-36.40,-3908.41,434.17,...,2,0.0,-39697.28,0.0,453,19087.78,6,0,4410.48,CONTINUA
4,1444378127,202105,1,0,0,30,25,-426.91,-4719.10,420.05,...,5,0.0,-53211.60,0.0,484,6592.48,4,0,914.94,None


## Ejercicio 1.1
* ¿Cuál es la nominalidad de cada clase?


In [27]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,622,825,160080
1,202102,831,1032,160292
2,202103,1039,951,161119
3,202104,955,1130,161333
4,202105,1134,0,0
5,202106,0,0,0


* ¿Cuál es la proporción del target?

## Guardar tabla en .csv

Para guardar a un `.csv` simplemente debe ejecutar la siguiente sentencia

In [28]:
%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)

,Success
